In [ ]:
import pandas as pd
import os
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from nltk import Tree,ParentedTree
%matplotlib inline
init_notebook_mode(connected=True) 

In [ ]:
df = pd.read_csv("Details.csv")

In [ ]:
df

In [ ]:
import datetime
datetime.datetime.strptime("9/9/2019","%m/%d/%Y")
dates = list(df['Date that the company has come'])
dates = [datetime.datetime.strptime(i,"%m/%d/%Y") for i in dates]

In [ ]:
dates = list(df['Date that the company has come'])
dates = [datetime.datetime.strptime(i,"%m/%d/%Y") for i in dates]
df['Date that the company has come'] = dates
df = df.sort_values(by='Date that the company has come')

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df

In [ ]:
trace1=go.Scatter(x=df['Date that the company has come'],y=df['CTC'],mode = 'lines+markers',
        hovertext = df['Name of company visited'],
        marker = dict(
                color = 'blue'
                ))
data = [trace1]
layout = dict(
    title='CTC of companies over time',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label='1m',
                     step='month',
                     stepmode='backward'),
                dict(count=6,
                     label='6m',
                     step='month',
                     stepmode='backward'),
                dict(count=1,
                    label='YTD',
                    step='year',
                    stepmode='todate'),
                dict(count=1,
                    label='1y',
                    step='year',
                    stepmode='backward'),
                dict(step='all')
            ])
        ),
        rangeslider=dict(
            visible = True
        ),
        type='date'
    )
)
fig = dict(data=data, layout=layout)
iplot(fig)
print('Move the slider to get deeper understanding of the CTC per day/week/month')



In [ ]:
ctc = df['CTC']
dates = df['Date that the company has come']


In [ ]:
from collections import Counter
tier = dict(Counter(df['Tier']))
data = [go.Bar(x=list(tier.keys()),y=list(tier.values()))]
layout = dict(title='Tier based division of companies',xaxis=dict(title="Tier"),
              yaxis = dict(title = "Number of companies"))
fig = go.Figure(data=data,layout=layout)
iplot(fig)

In [ ]:
d = {}
df = df.reset_index(drop=True)
for i in range(len(df)):
    d[df['Name of company visited'][i]] = float(df["CTC"][i].replace(',',''))
print(d)
import matplotlib.pyplot as plt
from wordcloud import WordCloud

wordcloud = WordCloud()
wordcloud.generate_from_frequencies(frequencies=d)

plt.figure(figsize = (10,10),facecolor=None)
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title('Companies that visited')
plt.show()

In [ ]:
from plotly.subplots import make_subplots
fig = make_subplots(rows=3, cols=2,subplot_titles=('July',"August","September","October","November"))
current=1
current_col = 1
for i in range(8,13):
    modified = df[(df['Date that the company has come']<datetime.datetime.strptime("2019-"+str(i)+"-01","%Y-%m-%d")) & (df['Date that the company has come']>datetime.datetime.strptime("2019-"+str(i-1)+"-01","%Y-%m-%d"))]
    #print('modified',modified)
    if(len(modified)):
        tier = dict(Counter(modified['Tier']))
        #print(current,current_col)
        fig.add_trace(go.Bar(x=list(tier.keys()),y=list(tier.values()),hovertext=list(tier.values())),row = current,col = current_col)
        current_col+=1
        #print(current,current_col)
        if(current_col==3):
            current_col=1
            current+=1
fig.update_layout(showlegend=False,height=1000,width=800,title="Tier division for different months") 


fig.update_yaxes(title_text="Number of companies", row=1, col=1)
fig.update_yaxes(title_text="Number of companies", row=1, col=2)
fig.update_yaxes(title_text="Number of companies", row=2, col=1)
fig.update_yaxes(title_text="Number of companies", row=2, col=2)
fig.update_yaxes(title_text="Number of companies", row=3, col=1)

fig.update_xaxes(title_text="Tier", row=1, col=1)
fig.update_xaxes(title_text="Tier", row=1, col=2)
fig.update_xaxes(title_text="Tier", row=2, col=1)
fig.update_xaxes(title_text="Tier", row=2, col=2)
fig.update_xaxes(title_text="Tier", row=3, col=1)
iplot(fig)

In [ ]:
import requests
from bs4 import BeautifulSoup
import wikipedia

def get_wiki_summary(page_soup):
    summary = ""
    paragraphs = page_soup.findAll()
    for paragraph in paragraphs:
        if paragraph.name == "p":
            summary += paragraph.text + " "
            #print("Adding summary")
        elif paragraph.name == "div" and paragraph.has_attr('class') and paragraph['class'][0] == 'toc':
            break

    #print(summary)
    return summary


def get_wiki_company_categories(page_soup):
    categories = []
    table_html = str(page_soup.find("table", {"class": "infobox vcard"}))
    if table_html == "None":
        #print("Returning none")
        return categories
    soup_table = BeautifulSoup(table_html, 'html.parser')
    td_list = soup_table.findAll("td", {"class": "category"})

    for item in td_list:
        if item.previous_sibling is not None and item.previous_sibling.text == "Industry":
            a_tag_list = BeautifulSoup(str(item), 'html.parser').findAll("a")
            for tag in a_tag_list:
                categories.append(tag.text)

    #print(categories)
    return categories

def get_wiki_company_suggestions(page_title):
    suggestions = wikipedia.search(page_title)
    for item in suggestions:
        if item.lower() == page_title.lower():
            categories_list = wikipedia.WikipediaPage(item).categories
            for category in categories_list:
                if "companies" or "company" in category:
                    return item 
            return None
    return None




def get_wiki_info(page_title):  
    response = requests.get("https://en.wikipedia.org/wiki/"+page_title)
    if response.status_code != 200:
        suggestion = get_wiki_company_suggestions(page_title)
        if suggestion == None:
            return [None, None]
        response = requests.get("https://en.wikipedia.org/wiki/"+suggestion)        

    page_html = response.content
    page_soup = BeautifulSoup(page_html, 'html.parser')
    
    summary = get_wiki_summary(page_soup)

    categories = get_wiki_company_categories(page_soup)

    return [summary, categories]


In [ ]:
company_names = list(df['Name of company visited'])
categories = []
for i in company_names:
    b = get_wiki_info(i)
    #print(i,end=' ')
    if(b[1]!=None and b[1]!=[]):
        print(b[1][0])
        categories.append(b[1][0])
    else:
        categories.append('Others')

In [ ]:
print(list(set(categories)))
df['category']=categories
df.to_csv('detail.csv',index=False)

In [ ]:
counts = (Counter(categories))
del counts['Others']
a = counts.most_common()
x_value=[]
y_value=[]
for i in a:
    x_value.append(i[0])
    y_value.append(i[1])
print(a)
data = [go.Bar(x=x_value,y=y_value)]
layout = dict(title='Different Company categories',xaxis=dict(title="Company categories"),
              yaxis = dict(title = "Number of companies"))
fig = go.Figure(data=data,layout=layout)
    
iplot(fig)

In [ ]:
df

In [ ]:
import random
random.randint(1,10)

In [ ]:
num_of_people = []
for i in range(len(df)):
    p = random.randint(1,15)
    num_of_people.append(random.randint(1,p))

In [ ]:
df['number']=num_of_people

In [ ]:
df.to_csv('detail.csv',index=False)

In [ ]:
values=[]
for i in range(8,13):
    modified = df[(df['Date that the company has come']<datetime.datetime.strptime("2019-"+str(i)+"-01","%Y-%m-%d")) & (df['Date that the company has come']>datetime.datetime.strptime("2019-"+str(i-1)+"-01","%Y-%m-%d"))]
    #print('modified',modified)
    if(len(modified)):
        tier = sum(modified['number'])
        values.append(tier)

        
data = [go.Bar(x=['July','August','September','October','November'],y=values)]
layout = dict(title='How many students are placed?',xaxis=dict(title="Month"),
              yaxis = dict(title = "Number of students"))
fig = go.Figure(data=data,layout=layout)
iplot(fig)

In [ ]:
values = []
for i in range(1,4):
    modified = df[df['Tier']==i]
    values.append(sum(modified['number']))

    
data = [go.Bar(x=['Tier-1','Tier-2','Tier-3'],y=values)]
layout = dict(title='Students composition based on company tiers',xaxis=dict(title="Month"),
              yaxis = dict(title = "Number of students"))
fig = go.Figure(data=data,layout=layout)
iplot(fig)


In [ ]:
data = [go.Box(x=df['CTC'],name='CTC offered',boxmean=True )]
layout = dict(title='CTC summary',xaxis=dict(title='CTC'))
fig = go.Figure(data=data,layout=layout)
iplot(fig)

In [ ]:
ctc = list(df['CTC'])
values =[]
results = ctc[2:]
results = [float(i.replace(',',''))/100000 for i in results]
for i in range(len(ctc)-1):
    values.append([float(ctc[i].replace(',',''))/100000,float(ctc[i+1].replace(',',''))/100000])

In [ ]:
values,results

In [ ]:
len(values)

In [ ]:
len(results)

In [ ]:
values.pop()

In [ ]:
import numpy as np
train_x = np.array(values[:70])
train_y = np.array(results[:70])
test_x = np.array(values[70:])
test_y = np.array(results[70:])

In [ ]:
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

n_features = 1
train_x = train_x.reshape((train_x.shape[0], train_x.shape[1], n_features))
# define model

In [ ]:
model = Sequential()
model.add(LSTM(7, activation='relu', input_shape=(2, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
model.fit(train_x, train_y, epochs=400, verbose=0)

In [ ]:
test_x = test_x.reshape(test_x.shape[0],test_x.shape[1],n_features)

In [ ]:
yhat = model.predict(test_x, verbose=1)


In [ ]:
mses = ((test_y-yhat)**2).mean(axis=1)
import math
print(math.sqrt(sum(mses)/len(mses)))
yhat_train = model.predict(train_x, verbose=1)
mses = ((train_y-yhat_train)**2).mean(axis=1)
print(math.sqrt(sum(mses)/len(mses)))

In [ ]:
train_x
x_axis =[i for i in range(len(train_x)+len(train_y))]
trace1=go.Scatter(x=x_axis,y=list(yhat_train.reshape(len(yhat_train)))+list(yhat.reshape(len(yhat))),name='predicted',mode = 'lines+markers')
trace2=go.Scatter(x=x_axis,y=list(train_y.reshape(len(train_y)))+list(test_y.reshape(len(test_y))),name='actual',mode = 'lines+markers')
fig = go.Figure(data=[trace1,trace2])
iplot(fig)

In [ ]:
train_y

In [ ]:
yhat.reshape(len(yhat))

In [ ]:
model.save('ctc_lstm.h5')